In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import tensorflow as tf

# **Day 3: Hands-on generative deep learning**

- Generative models
- Autoencoders
- Variatonal Autoencoders
- Generative Adversarial Networks

# Generative Models

> Generative models are probabilistic models that describe the distribution that generated a collection of training observations.

This means that generative models simulate how a dataset is generated, by sampling from them it is possible to generate new instances that look like they were sampled from the original data distribution.

- *Discriminative* models estimate \\( p(y|\mathbf{x}) \\), the probability of the label \\(y\\) given the input \\(\mathbf{x}\\).

- *Generative* models estimate (explicitly or implicitly) \\( p(\mathbf{x}) \\), the probability of observing \\(\mathbf{x}\\).

- *Class-conditional* generative models estimate \\( p(\mathbf{x}|y) \\), the probability of observing \\(\mathbf{x}\\) given the label \\(y\\).

Roughly speaking, a good generative model should
- generate examples that look as they were sampled from the original data distribution
- generate new examples, thus sufficiently different from the one sampled from the data distribution
- have a stochastic component to generate always new data

Generative models have several applications. They can be used for compression, denoising, inpainting, semi-supervised learning, synthetic data generation, and other tasks.

In the last years there have been huge advances in the field. A well known example is synthetic faces generation:
![](https://d3i71xaburhd42.cloudfront.net/a4d513cfc9d4902ef1a80198582f29b8ba46ac28/15-Figure2-1.png)
Check also [thispersondoesnotexist.com](https://thispersondoesnotexist.com/) and [gaugan](http://nvidia-research-mingyuliu.com/gaugan/), one of the [demo apps by NVIDIA](https://www.nvidia.com/en-us/research/ai-playground/). Have fun! :)

One major drawback of generative models is that progresses are not as easy to evaluate as for discriminative modeling. There are no clear metrics for performances comparison between models, especially when the quality of the output is largely subjective.

# Representation Learning

Most of the problems where generative modeling is applied have high-dimensional sample spaces (the input features space) with a high degree of conditional dependences between features.


Representation learning is a technique that tries to effciently retrieve the features that best describe the input observations exploiting pattern in the data. Assigning to each observation new learnt features, it creates a function that map the inputs to a lower dimensional space called the **latent** space.

Leveraging representation learning, generative models can overcome the high-dimensionality problem and reproduce common patterns found in the input distribution.

Guess which models excel in representation learning? :)

## Autoencoders

Autoencoders are sandwich-shaped neural networks composed by two parts:
![](https://www.pyimagesearch.com/wp-content/uploads/2020/02/keras_autoencoders_applications.png)
1. An **encoder** that compresses the inputs data in a lower dimensional latent representation (usually referred to as *codings*).
2. A **decoder** that converts back the codings from the latent space to the original domain (the output of the decoder is usually called *reconstruction*).

During **training**, the autoencoder tries to minimise the differences between the input data and the reconstruction of the input after it has been encoded. We can say that the autoencoder tries to **learn the identy function under some constraints**.



### Architecture
Clearly there is not an unique choice for the architecture of an autoencoder and it should be designed to fit the problem at hand. A best practice is to implement encoder and decoder in a symmetric way but still there are several things one can tune:

- Number of hidden layers (1, 2, ..3?, ..more?)
- Type of layers (Dense, convolution, LSTM, ..)
- Loss function (regression loss functions MSE, MAE, RMSE, .. or even classification losses like binary crossentropy!)

### Image reconstruction

Working with images there are a couple of things you may consider:

- Use the **binary crossentropy** as loss function: Instead of considering the reconstruction as a regression problem you can think abou it as a pixel-wise binary classification problem.

- Use **convolution** layers in the encoder and **transpose convolution** in the decoder.

Conv2D, "same", strides=2  |  Conv2DTranspose, "same", strides=2
:-------------------------:|:-------------------------:
![](https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/padding_strides.gif) | ![](https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/padding_strides_transposed.gif)

More gifs [here](https://github.com/vdumoulin/conv_arithmetic).
[Check the notebook](1.Autoencoder.ipynb)!


# Variational Autoencoders
VAEs have a similar architecture to classical autoencoders but a completly different nature. Indeed, they are *probabilistic models*:
<img style="float: left;" src="https://www.researchgate.net/profile/Abu_Kamruzzaman/publication/332540618/figure/fig2/AS:776663918997510@1562182516518/Variational-Autoencoder-17.jpg">

1. The reconstruction is nondeterministic

2. The model *generates new instances*


From the point of view of the implementation, the are only two parts that need to be changed to make an autoencoder variational:

1. The encoder's output.

2. The loss function.


![](https://miro.medium.com/max/3374/1*22cSCfmktNIwH5m__u2ffA.png)

### Encoder output

In vanilla autoencoders the encoder network maps inputs to points in the latent space. In a VAE an input is mapped first to a probability distribution: (usually) a multivariate Gaussian.

Multivariate Gaussians form a parametric family of distributions characterised by a mean vector and a covariance matrix (that can be assumed to be diagonal). Thus it suffices two vectors \\(\mathbf{\mu}\\) and \\(\mathbf{\sigma}\\) in \\(\mathbb{R}^\textsf{latent_dim}\\). 

For numerical stability, it is better to replace the standard deviation by its logarithm 
\\(\mathbf{\gamma}:= log(\mathbf{\sigma}^2)\\).

The encoder will first map an input \\(\mathbf{x}\\) to two vecotrs \\(\mathbf{\mu}\\) and \\(\mathbf{\gamma}\\), and then it will sample its output from \\(\mathcal{N}(\mathbf{z}|\mathbf{\mu}(\mathbf{x}),\mathbf{\gamma}(\mathbf{x})) \\).

### Latent Loss

In VAEs the cost function is the sum of two components:

- The usual **reconstruction loss** that forces the autoencoder to reproduce its inputs.

- The Kullback-Liebler **KL divergence** between \\(\mathcal{N}(\mathbf{z}|\mathbf{\mu(\mathbf{x})},\mathbf{\gamma}(\mathbf{x})) \\) and the standard normal distribution.

\\[ \mathcal{L} = - \frac{1}{2} \sum( 1 + \gamma_i - \text{exp}(\gamma_i) - \mu^2) \\]


[Check the notebook!](./2.VAEs.ipynb)

### Exercise

Modify the VAE implemented in the notebook replacing the dense layers by convolution (do not forget to use the transposed convolutions in the decoder).


1. First train it using latent dimension 2. How are the images compared to the dense VAE?


2. Increase the latent dimension to 50. What changed?


3. For each latent dimension, monitor how the histogram of the output of the encoder evolves during training. Do they look like standard normal distributions?

# Generative Adversarial Networks

GANs consist of two networks that are *adversary* of each other:

 - the **generator** generates samples intended to resemble the ones in the training dataset.
 
 - the **discriminator** inspects a sample and tells if it is real or fake.
 
 
 ![](https://www.oreilly.com/library/view/java-deep-learning/9781788997454/assets/2cf8b4f1-7163-4af1-aa4b-6066329d554a.png)

## The Generator

The generator takes as input a 1D tensor of random noise (generally Gaussian distributed) and produces data of the same format of the training data.

Similarly to the decoder it maps a latent space to the original domain.

In [1]:
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.models import Sequential

latent_dim = 10

generator = Sequential([
                Dense(100, activation="relu", input_shape=[latent_dim]),
                Dense(150, activation="relu"),
                Dense(28 * 28, activation="sigmoid"),
                Reshape([28, 28])])

## The Discriminator

The discriminator is a simple binary classifier. It has to predict the class of a given input: 1 if it is real, 0 if it is false.

In [4]:
from tensorflow.keras.layers import Flatten

discriminator = Sequential([
        Flatten(input_shape=[28, 28]),
        Dense(150, activation="relu"),
        Dense(100, activation="relu"),
        Dense(1, activation="sigmoid")
    ])

## Training a GAN

A training step happens in two phases:

1. **Supervised training of the discriminator** -  We feed to the discriminator a minibatch consisting of real and generated samples (50-50) and we perform a single train step by comparing its outputs to a target of 1s (real label) and 0s (fake labels).

2. **Supervised training of the generator** - We freeze the discriminator. After generating a minibatch of fake data we use the discriminator to produce a probability of the data to be real. We can train the generator parameters to produce outputs that are close to 1s as possible.

In [6]:
gan = Sequential([generator, discriminator])
# Compile the discriminator first
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
# Compile the whole GAN
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [2]:
def train_step(gan, x_batch, batch_size, latent_dim): # latent_dim and batch_size are not really necessary
    # Split gan
    generator, discriminator = gan.layers
    
    # Prepare the discriminator inputs and labels 
    z_noise    = tf.random.normal(shape=[batch_size, latent_dim])
    gen_images = generator(z_noise)
    x_disc     = tf.concat([gen_images, x_batch], axis=0)
    y_real     = tf.ones(batch_size)
    y_fake     = tf.zeros(batch_size)
    y_disc     = tf.concat([y_fake, y_real], axis=0)

    # 1. Training the discriminator
    discriminator.train_on_batch(x=x_disc, y=y_disc)
    
    # 2. Training the generator
    z_noise    = tf.random.normal(shape=[batch_size, latent_dim])
    gan.train_on_batch(x=z_noise, y=y_real)

## Deep Convolutional GANs

Some guidlines:

- Use batch normalisation in the generator (excluding the last layer)
- Use leaky relu activations where possible (not in the last layers!)
- Replace transpose convolution by upsampling + convolutions (why? check [this article](https://distill.pub/2016/deconv-checkerboard/))
- Use the hyperbolic tangent as last activation function in the generator (This requires rescaling the data between -1 and 1)


## GAN challenges
There are several known problems when training a GAN:

1. **Oscillating loss**

2. **Mode Collapse**

3. **Uninformative loss**

4. **Hyperparameters sensitivity**

## A lot of variants..

Check [this repository](https://github.com/hwalsuklee/tensorflow-generative-model-collections) for a comparison of different architectures/losses.

# Final Exercise

<img style="float: left;" src="https://www.researchgate.net/profile/Gerasimos_Spanakis/publication/328494719/figure/fig2/AS:685463685836800@1540438686532/LoGAN-architecture-where-x-denotes-the-real-image-c-the-class-label-z-the-noise.ppm">
Implement a [Auxiliary Classifier GANs](https://arxiv.org/abs/1610.09585) for the fashion_MNIST dataset.

The AC-GAN is an evolution of the vanilla GAN architecture that implements **class-conditional generative models**. This is achieved by:

1. adding an **auxiliary classifier** to the discriminator that predict the labels of the input images;

2. Concatenating an embedded representation of the label to the input noise of the generator (so it can generates class-conditional images) before the first convolution layer.

This architecture is highly sensible to the parameters and **tends to fall in mode collapse** on this dataset. This is an exercise focuses on the implementation, not on the quality of the generated images!

### Hints:
1. Modify the Discriminator to have a second output that predicts the label of the input image. Remeber, extra output = extra loss function for both the discriminator and the gan!


2. Add the label as a second input of the generator:
    - The label is encoded in an index, thus is categorical. To embedd a categorical varible in a vector space use an [Embedding Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) (if you are not sure how it works test in eager mode).
    - The embedded label (its dimension is up to you) has to be reshaped in a 3d tensor and concatenated to the input of the first Conv2D as a last feature map. If needed use a Dense layer between the embedding and the reshape.


3. If the above hints are not enough drop me a mail, I'm happy to discuss it with you.